*Created by Will Dinneen (willdinneen@gmail.com) for the PDRI-DevLab Junior Data Scientist Position @ UPenn*

## Introduction

- Here I will write my overall methodology and considerations, as well as a summary of key results

- In every section I will also explain my methodology more in depth before each code block

In [93]:
# # FOR PRINTING OUT TABLES!!

# df = grades_df[["person_id", "gr9_fall_math", "gr9_fall_hist"]].head(5)

# print(df.to_markdown(tablefmt="psql", index=None))

## Data Management

**case.csv** is the main dataset and reflects dates of arrest and disposition (trial or court appearance) during the period in which the program operated. The file also contains an indicator of whether the arrestee was referred to the intervention program for that arrest (i.e. whether they were treated), whether the person was rearrested while awaiting trial, the number of prior arrests at the time of program entry, and the arrest location. 

**demo.csv** contains demographic information about arrestees, including some who were not included in the program evaluation.

**prior_arrests.csv** reflects pre-period arrests among individuals in
*case.csv*;the pre-period ran from 2008-2011.

**grades.csv** includes 9th and 10th grade course grades for a subset of individuals in *case.csv*.

In [94]:
# Imports
import pandas as pd
import numpy as np

# Datasets
case_df = pd.read_csv('../case.csv')
demo_df = pd.read_csv('../demo.csv')
prior_arrests_df = pd.read_csv('../prior_arrests.csv')
grades_df = pd.read_csv('../grades.csv')

In [95]:
# 1. Recode it so that males are consistently coded as “M” and females are consistently coded as “F”.

print("Gender column values pre-recode:")
print(demo_df["gender"].unique())

# Gender column recoding
gender_recode = {
    'male': 'M',
    'female': 'F'
}

demo_df["gender"] = demo_df["gender"].replace(gender_recode)

print("\nGender column values post-recode:")
print(demo_df["gender"].unique())

Gender column values pre-recode:
['F' 'M' 'male' 'female']

Gender column values post-recode:
['F' 'M']


In [96]:
# 2. Merge the case and demo datasets together so that each row in the case dataset also contains the demographics of the defendant.

# Confirm Data Integrity
print("---- Checking Data Integrity ----\n")

missing_demo_values = demo_df["person_id"].isnull().sum()
missing_case_values = case_df["person_id"].isnull().sum()
print(f"Missing demographic values: {missing_demo_values}")
print(f"Missing case values: {missing_case_values}")

duplicate_demo_ids = demo_df["person_id"].duplicated().sum()
print(f"\nDuplicate demographic ids: {duplicate_demo_ids}")
# Check to see if duplicated IDs are duplicated rows
duplicate_ids = demo_df[demo_df["person_id"].duplicated(keep=False)]
duplicated_rows = duplicate_ids[duplicate_ids.duplicated(keep=False)]
print(f"Number of contradictory demographic ids: {len(duplicate_ids) - len(duplicated_rows)}")

# Dropping duplicates
demo_df = demo_df.drop_duplicates()

# Measure Differences Accross Demo & Case data
print("\n\n---- Comparing Demographic & Case Representation ----\n")

unique_case_ids = case_df["person_id"].unique()
unique_demo_ids = demo_df["person_id"].unique()
print("Number of unique persons in case data:")
print(len(unique_case_ids))

print("\nNumber of unique persons in demographic data:")
print(len(unique_demo_ids))

print("\nDifference in number of unique persons in demographic vs case data:")
print(len(unique_demo_ids) - len(unique_case_ids))

# Identify extra demo IDs & save to file for later investigation 
extra_demo_ids = set(unique_demo_ids) - set(unique_case_ids)
extra_demo_ids_df = pd.DataFrame(extra_demo_ids, columns=['person_id'])
extra_demo_ids_df.to_csv('./outputs/extra_demo_ids.csv', index=False)

# Merge the df
merged_df = case_df.merge(demo_df, on='person_id', how='left')

# Check for missing values
print(f"\nMissing values after merge:\n{merged_df.isnull().sum()}")

---- Checking Data Integrity ----

Missing demographic values: 0
Missing case values: 0

Duplicate demographic ids: 4721
Number of contradictory demographic ids: 0


---- Comparing Demographic & Case Representation ----

Number of unique persons in case data:
15353

Number of unique persons in demographic data:
15715

Difference in number of unique persons in demographic vs case data:
362

Missing values after merge:
caseid           0
person_id        0
arrest_date      0
dispos_date      0
treat            0
re_arrest        0
prior_arrests    0
address          0
race             0
gender           0
bdate            0
dtype: int64


In [97]:
# 3. For the purpose of this analysis, please restrict the data to only individuals who were arrested in Chicago.

# Check the format of the address column
address_format_check = merged_df["address"].str.contains(", ")
print(f"Number of addresses with correct format: {address_format_check.sum()}")
print(f"Number of addresses with incorrect format: {(~address_format_check).sum()}")

# Extract the city from the address column
merged_df["city"] = merged_df["address"].str.split(", ").str[1]

# Confirm that all cities are the same case
merged_df["city"] = merged_df["city"].str.lower()

# Check to see if there are any misspelled values
print(f"\nRepresented cities: {merged_df['city'].unique()}")

# Extract all rows with arrests in Chicago
chicago_df = merged_df[merged_df["city"] == "chicago"].copy()

print(f"\nNumber of cases in Chicago: {len(chicago_df)}")

Number of addresses with correct format: 26000
Number of addresses with incorrect format: 0

Represented cities: ['chicago' 'oak lawn' 'cicero']

Number of cases in Chicago: 25000


## Variable Creation

In [98]:
# 1. Create an age variable equal to the defendant’s age at the time of arrest for each case.

# Validate bdate & arrest date format
chicago_df['bdate'] = pd.to_datetime(chicago_df['bdate'], errors='coerce')
if chicago_df['bdate'].isnull().sum() > 0:
    print("Some birth dates have an incorrect format.")
else:
    print("All birth dates have the correct format.")
    
chicago_df['arrest_date'] = pd.to_datetime(chicago_df['arrest_date'], errors='coerce')
if chicago_df['arrest_date'].isnull().sum() > 0:
    print("Some arrest dates have an incorrect format.")
else:
    print("All arrest dates have the correct format.")

# Create age variable
chicago_df['age'] = np.floor((chicago_df['arrest_date'] - chicago_df['bdate']) / pd.Timedelta(days=365.25))

All birth dates have the correct format.
All arrest dates have the correct format.


In [99]:
grades_df

,person_id,gr9_fall_math,gr9_fall_sci,gr9_fall_eng,gr9_fall_hist,gr9_spring_math,gr9_spring_sci,gr9_spring_eng,gr9_spring_hist,gr10_fall_math,gr10_fall_sci,gr10_fall_eng,gr10_fall_hist,gr10_spring_math,gr10_spring_sci,gr10_spring_eng,gr10_spring_hist
0,1,D,NaN,A,A,A,D,A,NaN,D,A,A,D,A,A,A,A
1,5,D,NaN,A,A,A,D,D,A,D,A,A,A,D,F,A,A
2,8,NaN,D,F,A,A,D,D,D,A,D,NaN,D,D,D,A,A
3,10,NaN,A,A,A,NaN,A,A,D,A,A,A,A,A,A,A,F
4,11,NaN,A,NaN,A,F,NaN,D,D,D,D,A,F,A,NaN,D,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11246,19991,D,A,F,D,A,NaN,F,A,A,F,A,F,A,NaN,A,D
11247,19992,A,NaN,F,A,A,D,A,NaN,NaN,NaN,A,D,A,D,A,D
11248,19995,A,A,D,NaN,D,A,A,A,D,D,A,A,F,A,D,A
11249,19996,D,A,A,NaN,NaN,NaN,A,NaN,D,NaN,A,NaN,A,F,D,D


In [100]:
# 2. Please construct measures for 9th and 10th grade GPA for this target population.
grades_df

# Create the numeric grade scale
grade_scale_index = {
    "A": 4,
    "B": 3,
    "C": 2,
    "D": 1,
    "F": 0
}
grades_df = grades_df.replace(grade_scale_index)

# Create gr9 and gr10 GPA measures
def calculate_gpa(df, grade_level):
    relevant_columns = df.filter(like=grade_level)
    gpa_column = relevant_columns.mean(axis=1, skipna=True).round(3)
    return gpa_column
grades_df['gr9_gpa'] = calculate_gpa(grades_df, 'gr9')
grades_df['gr10_gpa'] = calculate_gpa(grades_df, 'gr10')

chicago_df

# Validate grades data
print("---- Validating Grades Data ----\n")
missing_grades_ids = grades_df['person_id'].isnull().sum()
print(f"Number of missing IDs in grade data: {missing_grades_ids}")
duplicate_grades_ids = grades_df["person_id"].duplicated().sum()
print(f"Duplicate grades ids: {duplicate_grades_ids}")

# Check if all relevant ids exist in both datasets
filtered_chicago_df = chicago_df[(chicago_df['age'] >= 18) & (chicago_df['age'] <= 24)]
relevant_chicago_ids = set(filtered_chicago_df['person_id'])
grade_ids = set(grades_df['person_id'])
missing_grades_ids = relevant_chicago_ids - grade_ids
print(f"\nNumber of relevant IDs in Chicago data not represented in Grade data: {len(missing_grades_ids)}")
missing_grades_data_ids = pd.DataFrame(missing_grades_ids, columns=['person_id'])
missing_grades_data_ids.to_csv('./outputs/missing_grades_data_ids.csv', index=False)
print("Missing grades data IDs saved to file: ./outputs/missing_grades_data_ids.csv")

# Add GPA measures to chicago_df
filtered_chicago_df = pd.merge(filtered_chicago_df, grades_df[['person_id', 'gr9_gpa', 'gr10_gpa']], on='person_id', how='left')
chicago_df = pd.merge(chicago_df, filtered_chicago_df[['caseid', 'gr9_gpa', 'gr10_gpa']], on='caseid', how='left')

---- Validating Grades Data ----

Number of missing IDs in grade data: 0
Duplicate grades ids: 0

Number of relevant IDs in Chicago data not represented in Grade data: 3006
Missing grades data IDs saved to file: ./outputs/missing_grades_data_ids.csv


In [101]:
chicago_df

,caseid,person_id,arrest_date,dispos_date,treat,re_arrest,prior_arrests,address,race,gender,bdate,city,age,gr9_gpa,gr10_gpa
0,57514,1,2012-01-04,2012-03-27,0,0,2,"1698 W 25TH PL, CHICAGO",WHITE,F,1985-07-03,chicago,26.0,NaN,NaN
1,39970,1,2012-07-11,2012-10-20,1,0,3,"4866 S CORNELL AVE, CHICAGO",WHITE,F,1985-07-03,chicago,27.0,NaN,NaN
2,88413,1,2013-04-04,2013-06-22,0,0,4,"2543 N WILLETTS CT, CHICAGO",WHITE,F,1985-07-03,chicago,27.0,NaN,NaN
3,40216,5,2012-03-31,2013-03-25,0,0,2,"4578 W MORSE AVE, CHICAGO",BLACK,M,1986-09-27,chicago,25.0,NaN,NaN
4,92255,6,2012-12-09,2013-11-09,0,0,3,"5111 S SANGAMON ST, CHICAGO",BLACK,M,1991-06-07,chicago,21.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,14458,19995,2013-02-16,2013-03-18,1,0,2,"1470 W EISENHOWER ASHLAND AV ER, CHICAGO",BLACK,M,1986-07-02,chicago,26.0,NaN,NaN
24996,15597,19996,2012-05-04,2012-08-04,0,0,5,"748 W 87TH ST S, CHICAGO",BLACK,F,1978-10-31,chicago,33.0,NaN,NaN
24997,2230,19997,2012-09-15,2012-10-02,0,0,2,"116 W GERMANIA PL, CHICAGO",ASIAN,M,1986-03-14,chicago,26.0,NaN,NaN
24998,21029,19999,2013-08-17,2014-10-19,0,0,2,"3032 E 136TH ST, CHICAGO",BLACK,M,1989-02-13,chicago,24.0,NaN,NaN


In [102]:
prior_arrests_df

,person_id,arrest_date
0,1,2008-06-14
1,1,2010-04-09
2,5,2009-06-07
3,5,2010-05-20
4,6,2008-03-16
...,...,...
43597,19997,2008-11-21
43598,19999,2009-07-26
43599,19999,2010-06-28
43600,20000,2008-06-01


In [103]:
# 3.a. Please reconstruct the variable (prior arrests) using the prior_arrests.csv file. 

# Validate prior arrests data
print("---- Validating Prior Arrests Data ----\n")
missing_arrest_ids = prior_arrests_df['person_id'].isnull().sum()
print(f"Number of missing IDs in prior arrests data: {missing_arrest_ids}")

# Validate date format
prior_arrests_df['arrest_date'] = pd.to_datetime(prior_arrests_df['arrest_date'], errors='coerce')
if prior_arrests_df['arrest_date'].isnull().sum() > 0:
    print("\nSome arrest dates have an incorrect format.")
else:
    print("All arrest dates have the correct format.")
    
# Confirm that there is no overlap so as not to double count
print(f"\nMost recent prior_arrest_df date: {prior_arrests_df['arrest_date'].max()}")
print(f"Oldest chicago_df date: {chicago_df['arrest_date'].min()}")

# Check if all arrest ids exist in both datasets
prior_arrests_ids = set(prior_arrests_df['person_id'])
chicago_ids = set(chicago_df['person_id'])
if prior_arrests_ids.issubset(chicago_ids):
    print("\nAll person IDs from prior_arrests_df exist in chicago_df")
else:
    print("\nNot all person IDs from prior_arrests_df exist in chicago_df")

# Reconstruct prior_arrests variable
prior_arrest_counts = prior_arrests_df.groupby('person_id').size().reset_index(name='reconstructed_prior_arrests')
chicago_df = pd.merge(chicago_df, prior_arrest_counts, on='person_id', how='left')
chicago_df['reconstructed_prior_arrests'] = chicago_df['reconstructed_prior_arrests'].fillna(0)
chicago_df['reconstructed_prior_arrests'] = chicago_df['reconstructed_prior_arrests'].astype(int)

# Add arrests that occured during the study period
chicago_df_sorted = chicago_df.sort_values(['person_id', 'arrest_date'])
chicago_df_sorted['reconstructed_prior_arrests'] = chicago_df_sorted.groupby('person_id').cumcount() + chicago_df_sorted['reconstructed_prior_arrests']
chicago_df.loc[chicago_df_sorted.index, 'reconstructed_prior_arrests'] = chicago_df_sorted['reconstructed_prior_arrests']

# 3.b. Please reconstruct this indicator (re_arrest).

# Calculate whether each row was rearrested
def calculate_re_arrests(row):
    other_cases = chicago_df_sorted[chicago_df_sorted['person_id'] == row['person_id']]
    subsequent_cases = other_cases[other_cases['arrest_date'] > row['arrest_date']]
    re_arrests = subsequent_cases[subsequent_cases['arrest_date'] < row['dispos_date']]
    return len(re_arrests) > 0

chicago_df_sorted['reconstructed_re_arrest'] = chicago_df_sorted.apply(calculate_re_arrests, axis=1)
chicago_df_sorted['reconstructed_re_arrest'] = chicago_df_sorted['reconstructed_re_arrest'].astype(int)

chicago_df.loc[chicago_df_sorted.index, 'reconstructed_re_arrest'] = chicago_df_sorted['reconstructed_re_arrest']

# 3.c. Please show that the variables you reconstructed are equal to the versions in the provided datasets.

print("\n\n---- Validating Reconstructed Variables ----\n")

# Check prior_arrests variable
prior_arrests_differences = chicago_df[chicago_df['prior_arrests'] != chicago_df['reconstructed_prior_arrests']]
print(f"Number of differenes between prior_arrests & reconstructed_prior_arrests: {len(prior_arrests_differences)}")

# Check re_arrest variable
re_arrest_differences = chicago_df[chicago_df['re_arrest'] != chicago_df['reconstructed_re_arrest']]
print(f"Number of differenes between re_arrest & reconstructed_re_arrest: {len(re_arrest_differences)}")

---- Validating Prior Arrests Data ----

Number of missing IDs in prior arrests data: 0
All arrest dates have the correct format.

Most recent prior_arrest_df date: 2011-12-31 00:00:00
Oldest chicago_df date: 2012-01-02 00:00:00

All person IDs from prior_arrests_df exist in chicago_df


---- Validating Reconstructed Variables ----

Number of differenes between prior_arrests & reconstructed_prior_arrests: 0
Number of differenes between re_arrest & reconstructed_re_arrest: 0


## Statistical Analysis

In [83]:
# 1. Describe the demographic characteristics of the study population based on the data available to you.

# 1.a. Are the treatment and control groups balanced? Please present your answer in the form of a table.

# 1.b. Choose one observable characteristic and visualize the difference between enrolled and not enrolled subjects.

In [84]:
# 3. Did participating in the program reduce the likelihood of re-arrest before disposition? Explain your answer and your methodology.

In [85]:
# 4. Using the data available to you, what recommendation would you make regarding who to serve?

## Conclusion